In [2]:
from entsoe.mappings import DOCUMENTTYPE
from torchgen.api.lazy import process_ir_type

API_TOKEN = ''

from entsoe import EntsoePandasClient
import pandas as pd
import time

client = EntsoePandasClient(api_key=API_TOKEN)

In [96]:
start = pd.Timestamp('20240101', tz='Europe/London')
end = pd.Timestamp('20241231T2359', tz='Europe/London')

df = client.query_generation(
    country_code='AT',
    start=start, end=end ,
    DOCUMENTTYPE = 'A71',
    nett=True,
)

KeyboardInterrupt: 

In [95]:
df.head(20)

AttributeError: 'NoneType' object has no attribute 'head'

In [17]:
countries = [
    "10YAT-APG------L",     # Austria
    "10YBE----------2",     # Belgium
    "10YBG-ESO-----R",      # Bulgaria
    '10YHR-HEP------M',     # Croatia
    '10YCY-1001A0003J',     # Cyprus
    "10YCZ-CEPS-----N",     # Czech Republic
    "10YDK-1--------W",     # Denmark
    '10Y1001A1001A39I'      # Estonia
    "10YFI-1--------U",     # Finland
    "10YFR-RTE------C",     # France
    "10YDE-VE-------2",     # Germany
    "10YGR-HTSO-----Y",     # Greece
    "10YHU-MAVIR----U",     # Hungary
    'IS',                   # Iceland
    "10YIE-1001A00010",     # Ireland
    "10YIT-GRTN-----B",     # Italy
    "10YLT-1001A0008Q",     # Lithuania
    "10YLV-1001A00074",     # Latvia
    '10YLU-CEGEDEL-NQ',     # Luxembourg
                            # Liechtenstein not available
    '10Y1001A1001A93C'      # Malta
    "10YNL----------L",     # Netherlands
    "10YNO-1--------2",     # Norway
    "10YPL-AREA-----S",     # Poland
    "10YPT-REN------W",     # Portugal
    "10YRO-TEL------P",     # Romania
    "10YSE-1--------K",     # Sweden
    "10YSI-ELES-----O",     # Slovenia
    '10YSK-SEPS-----K',     # Slovakia
    "10YES-REE------0",     # Spain
    "10YCH-SWISSGRIDZ"      # Switzerland (EFTA)
]


In [3]:
countries = [
    'AT',
    'BE',
    'BG',
    'HR',
    'CY',
    'CZ',
    'DK',
    'EE',
    'FI',
    'FR',
    'DE',
    'GR',
    'HU',
    'IS',
    'IT',
    'LV',
    'LT',
    'LU',
    'MT',
    'UK',
    'NL',
    'NO',
    'PL',
    'PT',
    'RO',
    'SK',
    'SI',
    'GB_NIR',
    'ES',
    'SE',
    'CH'
]

In [4]:
country_timezones = {
    'AT': 'Europe/Vienna',
    'BE': 'Europe/Brussels',
    'BG': 'Europe/Sofia',
    'HR': 'Europe/Zagreb',
    'CY': 'Asia/Nicosia',
    'CZ': 'Europe/Prague',
    'DK': 'Europe/Copenhagen',
    'EE': 'Europe/Tallinn',
    'FI': 'Europe/Helsinki',
    'FR': 'Europe/Paris',
    'DE': 'Europe/Berlin',
    'GR': 'Europe/Athens',
    'HU': 'Europe/Budapest',
    'IS': 'Atlantic/Reykjavik',
    'IT': 'Europe/Rome',
    'LV': 'Europe/Riga',
    'LT': 'Europe/Vilnius',
    'LU': 'Europe/Luxembourg',
    'MT': 'Europe/Malta',
    'UK': 'Europe/London',
    'NL': 'Europe/Amsterdam',
    'NO': 'Europe/Oslo',
    'PL': 'Europe/Warsaw',
    'PT': 'Europe/Lisbon',
    'RO': 'Europe/Bucharest',
    'SK': 'Europe/Bratislava',
    'SI': 'Europe/Ljubljana',
    'GB_NIR': 'Europe/Belfast',
    'ES': 'Europe/Madrid',
    'SE': 'Europe/Stockholm',
    'CH': 'Europe/Zurich'
}

In [5]:
psr_types = {
    "B01": "Biomass",
    "B02": "Fossil Brown coal/Lignite",
    "B03": "Fossil Coal-derived gas",
    "B04": "Fossil Gas",
    "B05": "Fossil Hard coal",
    "B06": "Fossil Oil",
    "B07": "Fossil Oil shale",
    "B08": "Fossil Peat",
    "B09": "Geothermal",
    "B10": "Hydro Pumped Storage",
    "B11": "Hydro Run-of-river and poundage",
    "B12": "Hydro Water Reservoir",
    "B13": "Marine",
    "B14": "Nuclear",
    "B15": "Other renewable",
    "B16": "Solar",
    "B17": "Waste",
    "B18": "Wind Offshore",
    "B19": "Wind Onshore",
    "B20": "Other"
}


In [ ]:
import os
import time
import pandas as pd

# Define time range
start_year = 2015
end_year = 2025  # This will process years 2015 up to 2024

# Folder to store individual country-year CSV files
data_dir = "data"
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

# Empty list to store DataFrames for later combination
data_list = []

# Assuming 'countries' is a list of country codes, for example:
# countries = ['AT', 'BE', 'DE', ...]
for country in countries:
    print(f"Fetching data for country: {country}")

    # Get the timezone for the current country, defaulting to Europe/Vienna if not found.
    tz = country_timezones.get(country, "Europe/Vienna")

    # Loop over years
    for year in range(start_year, end_year):
        # Create a filename for this country and year combination.
        file_name = os.path.join(data_dir, f"{country}_{year}.csv")

        # Check if file already exists. If so, load it.
        if os.path.exists(file_name):
            print(f"Data for {country} in {year} already exists. Loading from file.")
            try:
                # Make sure to parse the timestamp column if it is saved as such.
                df = pd.read_csv(file_name, parse_dates=["timestamp"])
            except Exception as e:
                print(f"Error reading {file_name}: {e}. Attempting to re-download data.")
                df = None
        else:
            df = None

        # If no valid data was loaded, fetch from the API.
        if df is None or df.empty:
            try:
                # Define the start and end timestamps (for one full year) using the country's timezone.
                start = pd.Timestamp(f"{year}-01-01", tz=tz)
                end = pd.Timestamp(f"{year}-12-31 23:59", tz=tz)

                print(f"Fetching generation data for {country} in period starting {year} using timezone {tz}...")
                # Replace client.query_generation with your actual API call.
                df = client.query_generation(
                    country_code=country,
                    start=start,
                    end=end,
                    psr_type=None,
                    nett=True
                )

                # If data exists, format and store it.
                if df is not None and not df.empty:
                    df["country"] = country
                    df["year"] = year

                    # Aggregate the data into daily totals (summing up the generation_MW values).
                    df = df.resample('D').sum()

                    # Save the aggregated DataFrame to CSV for later use.
                    df.to_csv(file_name, index=True, index_label="timestamp")
                    print(f"Saved daily aggregated data for {country} in {year} to {file_name}")
                else:
                    print(f"No data found for {country} in {year}.")
                    continue

                # Pause to respect API rate limits.
                time.sleep(1)

            except Exception as e:
                print(f"Error fetching data for {country} in {year}: {e}")
                time.sleep(5)  # Pause in case of an error, then continue to the next iteration.
                continue

        # Append the daily aggregated DataFrame (from file or API) to the list for later combination.
        data_list.append(df)

# Combine all individual DataFrames into one final DataFrame if any data was retrieved.
if data_list:
    final_df = pd.concat(data_list, ignore_index=False)
    final_csv_file = "entsoe_daily_generation_2015_2025.csv"
    final_df.to_csv(final_csv_file, index=True)
    print(f"Data fetching complete! Combined daily aggregated data saved as '{final_csv_file}'")
else:
    print("No data was retrieved.")

Fetching data for country: AT
Data for AT in 2015 already exists. Loading from file.
Data for AT in 2016 already exists. Loading from file.
Data for AT in 2017 already exists. Loading from file.
Data for AT in 2018 already exists. Loading from file.
Data for AT in 2019 already exists. Loading from file.
Data for AT in 2020 already exists. Loading from file.
Data for AT in 2021 already exists. Loading from file.
Data for AT in 2022 already exists. Loading from file.
Data for AT in 2023 already exists. Loading from file.
Data for AT in 2024 already exists. Loading from file.
Fetching data for country: BE
Data for BE in 2015 already exists. Loading from file.
Data for BE in 2016 already exists. Loading from file.
Data for BE in 2017 already exists. Loading from file.
Data for BE in 2018 already exists. Loading from file.
Data for BE in 2019 already exists. Loading from file.
Data for BE in 2020 already exists. Loading from file.
Data for BE in 2021 already exists. Loading from file.
Data 

In [38]:
# Set time range
start_year = 2015
end_year = 2025

# Empty list to store results
data_list = []

# Loop over countries
for country in countries:
    print(f"Fetching data for country: {country}")

    # Loop over years
    for year in range(start_year, end_year, 1):
        start = pd.Timestamp(f"{year}-01-01", tz="UTC")
        end = pd.Timestamp(f"{year+4}-12-31 23:59", tz="UTC")

        try:
            print(f"Fetching all generation data for {country} in {year}...")
            df = client.query_generation(
                country_code=country,
                start=start,
                end=end,
                psr_type=None,
                marketAgreementType="A01",  # Retrieve daily data directly
                nett=True
            )

            # If data exists, format and store it
            if df is not None and not df.empty:
                df.columns = ["generation_MW"]  # Ensure correct column name
                df["country"] = country
                df["year"] = year
                data_list.append(df)

            time.sleep(1)  # Avoid API rate limits

        except Exception as e:
            print(f"Error fetching data for {country} in {year}: {e}")
            time.sleep(5)  # Pause in case of error

# Combine all data into a single DataFrame
if data_list:
    final_df = pd.concat(data_list).reset_index()
    final_df.rename(columns={"index": "date"}, inplace=True)

    # Save to CSV
    final_df.to_csv("entsoe_daily_generation_2015_2025.csv", index=False)
    print("Data fetching complete! Saved as 'entsoe_daily_generation_2015_2025.csv'")
else:
    print("No data was retrieved.")


Fetching data for country: 10YAT-APG------L
Fetching all generation data for 10YAT-APG------L in 2015...


Connection Error, retrying in 0 seconds


Error fetching data for 10YAT-APG------L in 2015: HTTPSConnectionPool(host='web-api.tp.entsoe.eu', port=443): Max retries exceeded with url: /api?documentType=A75&processType=A16&in_Domain=10YAT-APG------L&securityToken=8112075a-2008-46a9-a0b7-d8e7f97e8a47&periodStart=201601010000&periodEnd=201701010000 (Caused by SSLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1000)')))
Fetching all generation data for 10YAT-APG------L in 2020...


Connection Error, retrying in 0 seconds


Error fetching data for 10YAT-APG------L in 2020: HTTPSConnectionPool(host='web-api.tp.entsoe.eu', port=443): Max retries exceeded with url: /api?documentType=A75&processType=A16&in_Domain=10YAT-APG------L&securityToken=8112075a-2008-46a9-a0b7-d8e7f97e8a47&periodStart=202201010000&periodEnd=202301010000 (Caused by SSLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1000)')))


...

Error fetching data for 10YSK-SEPS-----K in 2020: HTTPSConnectionPool(host='web-api.tp.entsoe.eu', port=443): Max retries exceeded with url: /api?documentType=A75&processType=A16&in_Domain=10YSK-SEPS-----K&securityToken=8112075a-2008-46a9-a0b7-d8e7f97e8a47&periodStart=202401010000&periodEnd=202501010000 (Caused by SSLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1000)')))
Fetching data for country: 10YES-REE------0
Fetching all generation data for 10YES-REE------0 in 2015...
Error fetching data for 10YES-REE------0 in 2015: Length mismatch: Expected axis has 20 elements, new values have 1 elements
Fetching all generation data for 10YES-REE------0 in 2020...
Error fetching data for 10YES-REE------0 in 2020: Length mismatch: Expected axis has 20 elements, new values have 1 elements
Fetching data for country: 10YCH-SWISSGRIDZ
Fetching all generation data for 10YCH-SWISSGRIDZ in 2015...


Connection Error, retrying in 0 seconds


Error fetching data for 10YCH-SWISSGRIDZ in 2015: HTTPSConnectionPool(host='web-api.tp.entsoe.eu', port=443): Max retries exceeded with url: /api?documentType=A75&processType=A16&in_Domain=10YCH-SWISSGRIDZ&securityToken=8112075a-2008-46a9-a0b7-d8e7f97e8a47&periodStart=201601010000&periodEnd=201701010000 (Caused by SSLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1000)')))
Fetching all generation data for 10YCH-SWISSGRIDZ in 2020...


Connection Error, retrying in 0 seconds


Error fetching data for 10YCH-SWISSGRIDZ in 2020: HTTPSConnectionPool(host='web-api.tp.entsoe.eu', port=443): Max retries exceeded with url: /api?documentType=A75&processType=A16&in_Domain=10YCH-SWISSGRIDZ&securityToken=8112075a-2008-46a9-a0b7-d8e7f97e8a47&periodStart=202401010000&periodEnd=202501010000 (Caused by SSLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1000)')))
No data was retrieved.


In [87]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd
import time
import datetime
import urllib3
import re

# Disable warnings for unverified HTTPS requests (for testing purposes only)
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

def parse_duration(duration_str):
    """
    Parse a duration string like 'PT60M' or 'PT1H' and return a timedelta.
    """
    match = re.match(r'PT(\d+)([HM])', duration_str)
    if match:
        value = int(match.group(1))
        unit = match.group(2)
        if unit == 'H':
            return datetime.timedelta(hours=value)
        elif unit == 'M':
            return datetime.timedelta(minutes=value)
    return None

# Define the list of years and country codes to query.
# (Ensure these values match data available from the API.)
years = [2023, 2024]  # Example: note that your sample XML has data for 2023.
countries = ['10YCZ-CEPS-----N']

# Your security token (if required; otherwise, leave it empty)
security_token = API_TOKEN  # Replace with your actual token

# Base API URL (without query parameters)
base_url = "https://web-api.tp.entsoe.eu//api"

# Optional: custom headers to mimic a browser
headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/90.0.4430.93 Safari/537.36"
    )
}

# List to store DataFrames for each API call
dataframes = []

for country in countries:
    for year in years:
        # Build periodStart and periodEnd for the request.
        # Format: YYYYMMDDhhmm (e.g., "202401010000" for Jan 1, 2024 00:00)
        periodStart = f"{year}01010000"
        periodEnd = f"{year}12312300"

        params = {
            "securityToken": security_token,
            "documentType": "A75",
            "processType": "A16",
            "in_Domain": country,
            "periodStart": periodStart,
            "periodEnd": periodEnd
        }

        print(f"Requesting data for {country} in {year}...")
        try:
            response = requests.get(base_url, params=params, headers=headers, verify=False)
            response.raise_for_status()
        except requests.RequestException as e:
            print(f"Error retrieving data for {country} in {year}: {e}")
            continue

        try:
            root = ET.fromstring(response.content)
        except ET.ParseError as e:
            print(f"Error parsing XML for {country} in {year}: {e}")
            continue

        # Handle the default namespace.
        ns = {}
        if root.tag.startswith("{"):
            ns_uri = root.tag.split("}")[0].strip("{")
            ns['ns'] = ns_uri
        else:
            ns['ns'] = ''

        records = []
        # Find all TimeSeries elements (using the namespace).
        timeseries_elements = root.findall('.//ns:TimeSeries', ns)
        if not timeseries_elements:
            print("No TimeSeries elements found in the XML.")

        for ts in timeseries_elements:
            # Locate the Period element (direct child of TimeSeries).
            period_elem = ts.find('ns:Period', ns)
            if period_elem is None:
                continue

            # Get the timeInterval inside Period and extract the start and end.
            time_interval = period_elem.find('ns:timeInterval', ns)
            if time_interval is None:
                continue
            start_elem = time_interval.find('ns:start', ns)
            end_elem = time_interval.find('ns:end', ns)
            if start_elem is None or end_elem is None:
                continue
            try:
                # Use the XML-provided start time as the baseline timestamp.
                period_start_dt = datetime.datetime.fromisoformat(start_elem.text.replace("Z", "+00:00"))
            except Exception as e:
                print(f"Error parsing period start time '{start_elem.text}': {e}")
                continue

            # Resolution is a direct child of Period.
            resolution_elem = period_elem.find('ns:resolution', ns)
            resolution_str = resolution_elem.text if resolution_elem is not None else None
            interval_td = parse_duration(resolution_str) if resolution_str else None

            # Find all Point elements directly under Period.
            points = period_elem.findall('ns:Point', ns)
            for point in points:
                pos_elem = point.find('ns:position', ns)
                quantity_elem = point.find('ns:quantity', ns)
                if pos_elem is None or quantity_elem is None:
                    continue
                try:
                    pos = int(pos_elem.text)
                    quantity = float(quantity_elem.text)
                except Exception as e:
                    print("Error parsing position or quantity:", e)
                    continue

                # Calculate the timestamp for this point.
                timestamp = period_start_dt + (interval_td * (pos - 1)) if interval_td else period_start_dt
                record = {
                    "timestamp": timestamp,
                    "quantity": quantity,
                    "resolution": resolution_str,
                    "in_Domain": country,
                    "year": year,
                    "period_start": start_elem.text,
                    "period_end": end_elem.text
                }
                records.append(record)

        if records:
            df_temp = pd.DataFrame(records)
            dataframes.append(df_temp)
        else:
            print(f"No valid records found for {country} in {year}.")

        # Pause briefly to respect API rate limits.
        time.sleep(0.5)

if dataframes:
    combined_df = pd.concat(dataframes, ignore_index=True)
    print("Combined DataFrame:")
    print(combined_df.head())
else:
    print("No data was retrieved from the API.")


Requesting data for 10YCZ-CEPS-----N in 2023...
Requesting data for 10YCZ-CEPS-----N in 2024...
Combined DataFrame:
                  timestamp  quantity resolution         in_Domain  year  \
0 2023-01-01 00:00:00+00:00     266.0      PT60M  10YCZ-CEPS-----N  2023   
1 2023-01-01 01:00:00+00:00     266.0      PT60M  10YCZ-CEPS-----N  2023   
2 2023-01-01 02:00:00+00:00     266.0      PT60M  10YCZ-CEPS-----N  2023   
3 2023-01-01 03:00:00+00:00     266.0      PT60M  10YCZ-CEPS-----N  2023   
4 2023-01-01 04:00:00+00:00     266.0      PT60M  10YCZ-CEPS-----N  2023   

        period_start         period_end  
0  2023-01-01T00:00Z  2023-12-31T23:00Z  
1  2023-01-01T00:00Z  2023-12-31T23:00Z  
2  2023-01-01T00:00Z  2023-12-31T23:00Z  
3  2023-01-01T00:00Z  2023-12-31T23:00Z  
4  2023-01-01T00:00Z  2023-12-31T23:00Z  


In [88]:
combined_df.head(48)

,timestamp,quantity,resolution,in_Domain,year,period_start,period_end
0,2023-01-01 00:00:00+00:00,266.0,PT60M,10YCZ-CEPS-----N,2023,2023-01-01T00:00Z,2023-12-31T23:00Z
1,2023-01-01 01:00:00+00:00,266.0,PT60M,10YCZ-CEPS-----N,2023,2023-01-01T00:00Z,2023-12-31T23:00Z
2,2023-01-01 02:00:00+00:00,266.0,PT60M,10YCZ-CEPS-----N,2023,2023-01-01T00:00Z,2023-12-31T23:00Z
3,2023-01-01 03:00:00+00:00,266.0,PT60M,10YCZ-CEPS-----N,2023,2023-01-01T00:00Z,2023-12-31T23:00Z
4,2023-01-01 04:00:00+00:00,266.0,PT60M,10YCZ-CEPS-----N,2023,2023-01-01T00:00Z,2023-12-31T23:00Z
5,2023-01-01 05:00:00+00:00,267.0,PT60M,10YCZ-CEPS-----N,2023,2023-01-01T00:00Z,2023-12-31T23:00Z
6,2023-01-01 06:00:00+00:00,268.0,PT60M,10YCZ-CEPS-----N,2023,2023-01-01T00:00Z,2023-12-31T23:00Z
7,2023-01-01 07:00:00+00:00,267.0,PT60M,10YCZ-CEPS-----N,2023,2023-01-01T00:00Z,2023-12-31T23:00Z
8,2023-01-01 08:00:00+00:00,268.0,PT60M,10YCZ-CEPS-----N,2023,2023-01-01T00:00Z,2023-12-31T23:00Z
9,2023-01-01 09:00:00+00:00,268.0,PT60M,10YCZ-CEPS-----N,2023,2023-01-01T00:00Z,2023-12-31T23:00Z
